In [1]:
import pandas as pd
import numpy as np

from PIL import Image
import os
import scipy
import cv2

from tqdm import tqdm
from time import time
import json 

import warnings
warnings.simplefilter("ignore")

In [2]:
with open('config.json') as f_in:
    config = json.load(f_in)

In [3]:
config['stat_vignettes_path'] = 'vignettes/stat_vignettes.csv'
config['stat_vignettes_col' ] = ['file', 'wood_size', 'w_mean_R', 'w_mean_G', 'w_mean_B', 'nw_mean_R', 'nw_mean_G', 'nw_mean_B', 'w_std_R', 'w_std_G', 'w_std_B', 'nw_std_R', 'nw_std_G', 'nw_std_B']

In [85]:
config

{'mnhc_path': 'donnees/MNHC/',
 'ortho_path': {'35': {'2020': 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/',
   '2017': 'donnees/BDORTHO/BDORTHO_2-0_RVB-0M50_JP2-E080_LAMB93_D035_2017-01-01/BDORTHO/1_DONNEES_LIVRAISON_2019-01-00118/BDO_RVB_0M50_JP2-E080_LAMB93_D35-2017/',
   '2012': 'donnees/BDORTHO/BDORTHO_2-0_RVB-0M50_JP2-E080_LAMB93_D035_2012-01-01/BDORTHO/1_DONNEES_LIVRAISON_2015-02-00365/BDO_RVB_0M50_JP2-E080_LAMB93_D35-2012/'},
  '49': {'2020': 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D049_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-12-00085/OHR_RVB_0M20_JP2-E080_LAMB93_D49-2020/'}},
 'irc_path': {'35': {'2020': 'donnees/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00110/OHR_IRC_0M20_JP2-E080_LAB93_D35-2020/'},
  '49': {'2020': 'donnees/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D049_2020-01-01/ORTHOHR

In [4]:
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [46]:
def std_convoluted(im, N):
    """
    Calculate the standard deviation using a convolution operation.

    Parameters:
    - im (numpy.ndarray): Input image as a NumPy array.
    - N (int): Radius of the convolution kernel.

    Returns:
    - numpy.ndarray: Standard deviation image.
    """
    # Calculate squared image and initialize kernel
    im2 = im**2
    ones = np.ones(im.shape)
    kernel = np.ones((2*N+1, 2*N+1))

    # Convolve the image and squared image with the kernel
    s = scipy.signal.convolve2d(im, kernel, mode="same")
    s2 = scipy.signal.convolve2d(im2, kernel, mode="same")
    ns = scipy.signal.convolve2d(ones, kernel, mode="same")

    # Calculate the standard deviation image
    return np.sqrt((s2 - s**2 / ns) / ns)

def my_func(path_to_orthophoto_rgb):
    
    Image.MAX_IMAGE_PIXELS = None
    
    rgb_name = os.path.basename(path_to_orthophoto_rgb)
    dept = rgb_name[:2]
    year = rgb_name[3:7]
    path_to_orthophoto_irc = config["irc_path"][dept][year] + rgb_name[:config['irc_pos']] + '-IRC' + rgb_name[config['irc_pos']:]# config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:] # A Changer plus tard
    # mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(path_to_orthophoto_rgb[config['rgb_coordinates_pos']:config['rgb_coordinates_pos']+8]).replace('-', '_') + ".tif"
    
    # position
    rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
    rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
    mnhc_dir_dept = config['mnhc_path']+dept+'_'+year+'/'
    
    # images IRC et RGB
    # ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
    # ouverture
    ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
    ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA) 
    ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where=(ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
    ortho_irc = None 
    
    if not os.path.exists(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/')
    if not os.path.exists(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/')

    if os.path.exists(config['stat_vignettes_path']):
        df = pd.read_csv(config['stat_vignettes_path'], index_col="Unnamed: 0")
    else:
        df = pd.DataFrame(columns=config['stat_vignettes_col'])

    # display(df)
    
    # i = 0
    # vignette_data = []
    for mnhc_x in tqdm(range(0,5)):
        for mnhc_y in range(0,5):
            mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
            
            if os.path.exists(mnhc_file):
                # print(mnhc_file)
                mnhc = np.asarray(Image.open(mnhc_file))
                w, h = mnhc.shape
                mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                     np.logical_or(mnhc > 5., 
                                                                   std_convoluted(mnhc,N=10) > .5)
                                                    ),
                                      ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
                .astype('uint8')

                
                for x in range(0, w-config['img_size']+1, config['img_size']):
                    for y in range(0, h-config['img_size']+1, config['img_size']):
                        # print(x, y)
                        crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+config['img_size'], mnhc_x*w+x: mnhc_x*w+x+config['img_size'], :]
                        crop_mask = mask[y:y+config['img_size'], x:x+config['img_size']]

                        # filtre rgb
                        # enregistrement
                        pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                        
                        cv2.imwrite(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                        cv2.imwrite(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask*255)

                        # Calculate characteristics of the RGB crop and add to vignette_data
                        mask_2 = np.stack((crop_mask,)*3, axis=-1)
                        non_mask = np.logical_not(mask_2)
                        res = [pos, crop_mask.sum()] + \
                        np.mean(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.mean(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist()
                        # vignette_data.append(res)
                        
                        #append list to DataFrame
                        df.loc[len(df) + len(set(range(df.index[-1]))-set(df.index))] = res
                
                # Si besoin de plus d'images, commenter le break
                break
    
    df.drop_duplicates(subset='file').to_csv(config['stat_vignettes_path'])
    # Convert the vignette data to a Pandas DataFrame and return
    return df
                

In [50]:
path = 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/35-2020-0310-6780-LA93-0M20-E080.jp2'
df = my_func(path)

  0%|                                    | 0/5 [00:00<?, ?it/s]

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_305000_6781000,44665,49.817732,62.395007,50.272898,72.778496,79.152460,60.173830,16.147061,16.278274,11.471902,21.932427,20.567786,14.379507
1,35_305000_6781128,24762,44.115863,58.678580,47.686092,82.456050,99.597489,71.074999,13.473296,14.724916,10.345457,21.296138,17.277575,18.104901
2,35_305000_6781256,23237,45.433102,59.839265,48.430391,92.960874,106.022979,80.356675,16.232044,16.493535,10.681222,36.763966,35.429339,29.701126
3,35_305000_6781384,22756,50.127219,62.966602,49.803261,69.991889,82.777887,59.352174,19.416147,18.474566,12.218301,20.434826,18.459442,11.694978
4,35_305000_6781512,8240,59.035194,72.758981,56.212864,76.377042,91.536495,63.268849,21.260137,19.966213,12.459335,20.583157,15.015475,9.347617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,35_310768_6780256,26375,55.927962,64.056910,52.213081,99.500728,94.670208,77.765277,21.838344,19.436566,13.745432,41.160810,32.297688,30.446841
535,35_310768_6780384,14951,60.322721,67.733061,54.428935,97.042839,94.608224,79.189859,20.387251,18.954500,13.641429,37.581204,31.435458,31.010447
536,35_310768_6780512,12212,66.023829,71.467409,57.342286,88.823025,86.102393,71.465888,23.068565,20.029768,15.566703,33.394956,26.090063,23.217838
537,35_310768_6780640,37549,50.974087,59.566646,47.600495,69.390074,71.543038,57.353164,17.258172,15.891011,10.476188,23.424131,18.607226,13.289627


 20%|█████▌                      | 1/5 [00:07<00:31,  7.95s/it]

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_305000_6781000,44665,49.817732,62.395007,50.272898,72.778496,79.152460,60.173830,16.147061,16.278274,11.471902,21.932427,20.567786,14.379507
1,35_305000_6781128,24762,44.115863,58.678580,47.686092,82.456050,99.597489,71.074999,13.473296,14.724916,10.345457,21.296138,17.277575,18.104901
2,35_305000_6781256,23237,45.433102,59.839265,48.430391,92.960874,106.022979,80.356675,16.232044,16.493535,10.681222,36.763966,35.429339,29.701126
3,35_305000_6781384,22756,50.127219,62.966602,49.803261,69.991889,82.777887,59.352174,19.416147,18.474566,12.218301,20.434826,18.459442,11.694978
4,35_305000_6781512,8240,59.035194,72.758981,56.212864,76.377042,91.536495,63.268849,21.260137,19.966213,12.459335,20.583157,15.015475,9.347617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,35_311768_6780256,65473,48.545660,58.121989,46.242558,41.968254,51.936508,44.571429,18.195732,17.456877,11.831166,19.504959,16.609304,12.501066
584,35_311768_6780384,65144,47.285322,57.691852,46.226176,39.721939,47.780612,41.864796,17.066167,16.956623,11.757379,18.226795,17.050580,11.341390
585,35_311768_6780512,64329,46.916399,57.677377,46.634659,65.331400,67.590721,54.779619,15.858517,15.811471,11.407279,32.147720,25.070998,18.103298
586,35_311768_6780640,65363,43.203510,55.141502,45.805241,43.225434,54.728324,45.121387,11.334820,12.016217,9.373565,11.240978,11.615276,8.672743


 40%|███████████▏                | 2/5 [00:15<00:23,  7.88s/it]

...

 60%|████████████████▊           | 3/5 [00:23<00:15,  7.85s/it]

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_305000_6781000,44665,49.817732,62.395007,50.272898,72.778496,79.152460,60.173830,16.147061,16.278274,11.471902,21.932427,20.567786,14.379507
1,35_305000_6781128,24762,44.115863,58.678580,47.686092,82.456050,99.597489,71.074999,13.473296,14.724916,10.345457,21.296138,17.277575,18.104901
2,35_305000_6781256,23237,45.433102,59.839265,48.430391,92.960874,106.022979,80.356675,16.232044,16.493535,10.681222,36.763966,35.429339,29.701126
3,35_305000_6781384,22756,50.127219,62.966602,49.803261,69.991889,82.777887,59.352174,19.416147,18.474566,12.218301,20.434826,18.459442,11.694978
4,35_305000_6781512,8240,59.035194,72.758981,56.212864,76.377042,91.536495,63.268849,21.260137,19.966213,12.459335,20.583157,15.015475,9.347617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,35_313768_6780256,65477,56.355591,66.407196,52.098233,66.711864,75.932203,62.559322,18.634110,18.284673,13.174091,20.539380,19.104409,17.660903
682,35_313768_6780384,32591,66.712804,74.260808,58.023381,157.415541,147.071027,129.015784,23.983346,21.283840,16.026230,42.318915,33.256850,35.260658
683,35_313768_6780512,8789,80.773808,83.876323,68.070543,161.565404,151.384267,135.907854,29.353229,23.514166,17.421966,44.440901,36.103276,37.282713
684,35_313768_6780640,9049,73.118135,80.187645,64.549011,166.628746,157.493158,141.782729,28.373008,24.177872,16.268409,45.928371,37.137834,39.560581


 80%|██████████████████████▍     | 4/5 [00:31<00:07,  7.83s/it]

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_305000_6781000,44665,49.817732,62.395007,50.272898,72.778496,79.152460,60.173830,16.147061,16.278274,11.471902,21.932427,20.567786,14.379507
1,35_305000_6781128,24762,44.115863,58.678580,47.686092,82.456050,99.597489,71.074999,13.473296,14.724916,10.345457,21.296138,17.277575,18.104901
2,35_305000_6781256,23237,45.433102,59.839265,48.430391,92.960874,106.022979,80.356675,16.232044,16.493535,10.681222,36.763966,35.429339,29.701126
3,35_305000_6781384,22756,50.127219,62.966602,49.803261,69.991889,82.777887,59.352174,19.416147,18.474566,12.218301,20.434826,18.459442,11.694978
4,35_305000_6781512,8240,59.035194,72.758981,56.212864,76.377042,91.536495,63.268849,21.260137,19.966213,12.459335,20.583157,15.015475,9.347617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,35_314768_6780256,11450,68.513100,77.387686,63.444629,103.895278,106.845431,86.987002,26.189128,23.254675,15.611365,37.380925,29.104237,30.509286
731,35_314768_6780384,1825,73.690959,82.791233,64.610411,79.570482,96.888701,67.786615,24.280571,21.961055,16.465456,14.837685,12.939336,10.866015
732,35_314768_6780512,9684,66.435254,76.957352,59.199917,78.790768,93.139565,67.427648,22.697835,21.426221,15.388790,22.373906,19.558260,17.353923
733,35_314768_6780640,8994,64.354681,75.127641,58.915944,78.082700,93.481730,67.383343,22.304074,21.025809,13.748360,22.086248,16.074489,16.189976


100%|████████████████████████████| 5/5 [00:39<00:00,  7.86s/it]


In [51]:
df#.drop('Unnamed: 0.1', axis=1).to_csv(config['stat_vignettes_path'])

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_305000_6781000,44665,49.817732,62.395007,50.272898,72.778496,79.152460,60.173830,16.147061,16.278274,11.471902,21.932427,20.567786,14.379507
1,35_305000_6781128,24762,44.115863,58.678580,47.686092,82.456050,99.597489,71.074999,13.473296,14.724916,10.345457,21.296138,17.277575,18.104901
2,35_305000_6781256,23237,45.433102,59.839265,48.430391,92.960874,106.022979,80.356675,16.232044,16.493535,10.681222,36.763966,35.429339,29.701126
3,35_305000_6781384,22756,50.127219,62.966602,49.803261,69.991889,82.777887,59.352174,19.416147,18.474566,12.218301,20.434826,18.459442,11.694978
4,35_305000_6781512,8240,59.035194,72.758981,56.212864,76.377042,91.536495,63.268849,21.260137,19.966213,12.459335,20.583157,15.015475,9.347617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,35_314768_6780256,11450,68.513100,77.387686,63.444629,103.895278,106.845431,86.987002,26.189128,23.254675,15.611365,37.380925,29.104237,30.509286
731,35_314768_6780384,1825,73.690959,82.791233,64.610411,79.570482,96.888701,67.786615,24.280571,21.961055,16.465456,14.837685,12.939336,10.866015
732,35_314768_6780512,9684,66.435254,76.957352,59.199917,78.790768,93.139565,67.427648,22.697835,21.426221,15.388790,22.373906,19.558260,17.353923
733,35_314768_6780640,8994,64.354681,75.127641,58.915944,78.082700,93.481730,67.383343,22.304074,21.025809,13.748360,22.086248,16.074489,16.189976


In [56]:
path_to_orthophoto_rgb = 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/' + \
    '35-2020-0310-6780-LA93-0M20-E080.jp2'

Image.MAX_IMAGE_PIXELS = None

rgb_name = os.path.basename(path_to_orthophoto_rgb)
dept = rgb_name[:2]
year = rgb_name[3:7]
path_to_orthophoto_irc = config["irc_path"][dept][year] + rgb_name[:config['irc_pos']] + '-IRC' + rgb_name[config['irc_pos']:]# config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:] # A Changer plus tard
# mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(path_to_orthophoto_rgb[config['rgb_coordinates_pos']:config['rgb_coordinates_pos']+8]).replace('-', '_') + ".tif"

# position
rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
mnhc_dir_dept = config['mnhc_path']+dept+'_'+year+'/'

# images IRC et RGB
# ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
# ouverture
ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA) 
ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where=(ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
ortho_irc = None 

In [83]:


if not os.path.exists(config['rgb_vignettes_path'] + f'test_rgb_{rgb_x}_{rgb_y}/'):
    os.makedirs(config['rgb_vignettes_path'] + f'test_rgb_{rgb_x}_{rgb_y}/')
if not os.path.exists(config['mask_vignettes_path'] + f'test_mask_{rgb_x}_{rgb_y}/'):
    os.makedirs(config['mask_vignettes_path'] + f'test_mask_{rgb_x}_{rgb_y}/')

# if os.path.exists(config['stat_vignettes_path']):
#     df = pd.read_csv(config['stat_vignettes_path'], index_col="Unnamed: 0")
# else:
#     df = pd.DataFrame(columns=config['stat_vignettes_col'])

In [89]:
df = pd.DataFrame(columns=config['stat_vignettes_col'])
display(df)

# i = 0
# vignette_data = []
for mnhc_x in tqdm(range(0,5)):
    for mnhc_y in range(0,5):
        mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
        
        if os.path.exists(mnhc_file):
            # print(mnhc_file)
            mnhc = np.asarray(Image.open(mnhc_file))
            w, h = mnhc.shape
            mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                 np.logical_or(mnhc > 5., 
                                                               std_convoluted(mnhc,N=10) > .5)
                                                ),
                                  ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
            .astype('uint8')

            
            for x in range(0, w-(config['img_size']+config['border']*2)+1, config['img_size']+config['border']*2):
                for y in range(0, h-(config['img_size']+config['border']*2)+1, config['img_size']+config['border']*2):
                    
                    # print(f"rgb: {mnhc_y*h+y}, {mnhc_y*h+y+config['img_size']+config['border']*2}, {mnhc_x*w+x}, {mnhc_x*w+x+config['img_size']+config['border']*2}")
                    # print(f"mask: {y}, {y+config['img_size']+config['border']*2}, {x}, {x+config['img_size']+config['border']*2}")
                    # print('')
                    
                    crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+config['img_size']+config['border']*2, mnhc_x*w+x: mnhc_x*w+x+config['img_size']+config['border']*2, :]
                    crop_mask = mask[y:y+config['img_size']+config['border']*2, x:x+config['img_size']+config['border']*2]

                    # filtre rgb
                    # enregistrement
                    pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                    
                    cv2.imwrite(config['rgb_vignettes_path'] + f'test_rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                    cv2.imwrite(config['mask_vignettes_path'] + f'test_mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask*255)

                    # Calculate characteristics of the RGB crop and add to vignette_data
                    mask_2 = np.stack((crop_mask,)*3, axis=-1)
                    non_mask = np.logical_not(mask_2)

                    print(mask_2.shape, crop_rgb.shape)
                    res = [pos, crop_mask.sum()] + \
                    np.mean(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                    np.mean(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
                    np.std(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                    np.std(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist()
                    # vignette_data.append(res)
                    
                    #append list to DataFrame
                    df.loc[len(df) + len(set(range(df.index[-1]))-set(df.index))
                    if len(df.index) > 0
                    else 0] = res
            
            # Si besoin de plus d'images, commenter le break
            break
            

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B


  0%|                                    | 0/5 [00:00<?, ?it/s]

rgb: 0, 288, 0, 288
mask: 0, 288, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 0, 288
mask: 288, 576, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 0, 288
mask: 576, 864, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 0, 288
mask: 864, 1152, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 0, 288
mask: 1152, 1440, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 0, 288
mask: 1440, 1728, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 288, 576
mask: 0, 288, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 288, 576
mask: 288, 576, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 288, 576
mask: 576, 864, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 288, 576
mask: 864, 1152, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 288, 576
mask: 1152, 1440, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 288, 576
mask: 1440, 1728, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 576, 864
mask: 0, 288, 576, 864

(288, 288, 3) (288, 288, 3)
rg

 20%|█████▌                      | 1/5 [00:07<00:30,  7.71s/it]

rgb: 288, 576, 1440, 1728
mask: 288, 576, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 1440, 1728
mask: 576, 864, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 1440, 1728
mask: 864, 1152, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 1440, 1728
mask: 1152, 1440, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 1440, 1728
mask: 1440, 1728, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 2000, 2288
mask: 0, 288, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 2000, 2288
mask: 288, 576, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 2000, 2288
mask: 576, 864, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 2000, 2288
mask: 864, 1152, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 2000, 2288
mask: 1152, 1440, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 2000, 2288
mask: 1440, 1728, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 2288, 2576
mask: 0, 288, 288, 576

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 2288, 2576
mask

...

rgb: 0, 288, 5440, 5728
mask: 0, 288, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 5440, 5728
mask: 288, 576, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 5440, 5728
mask: 576, 864, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 5440, 5728
mask: 864, 1152, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 5440, 5728
mask: 1152, 1440, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 5440, 5728
mask: 1440, 1728, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 6000, 6288
mask: 0, 288, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 6000, 6288
mask: 288, 576, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 6000, 6288
mask: 576, 864, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 6000, 6288
mask: 864, 1152, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 6000, 6288
mask: 1152, 1440, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 6000, 6288
mask: 1440, 1728, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 6288, 6576
mask

 80%|██████████████████████▍     | 4/5 [00:31<00:07,  7.82s/it]

rgb: 0, 288, 7440, 7728
mask: 0, 288, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 7440, 7728
mask: 288, 576, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 7440, 7728
mask: 576, 864, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 7440, 7728
mask: 864, 1152, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 7440, 7728
mask: 1152, 1440, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 7440, 7728
mask: 1440, 1728, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 8000, 8288
mask: 0, 288, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 288, 576, 8000, 8288
mask: 288, 576, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 8000, 8288
mask: 576, 864, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 8000, 8288
mask: 864, 1152, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 8000, 8288
mask: 1152, 1440, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 8000, 8288
mask: 1440, 1728, 0, 288

(288, 288, 3) (288, 288, 3)
rgb: 0, 288, 8288, 8576
mask

100%|████████████████████████████| 5/5 [00:38<00:00,  7.79s/it]

(288, 288, 3) (288, 288, 3)
rgb: 576, 864, 9440, 9728
mask: 576, 864, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 864, 1152, 9440, 9728
mask: 864, 1152, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1152, 1440, 9440, 9728
mask: 1152, 1440, 1440, 1728

(288, 288, 3) (288, 288, 3)
rgb: 1440, 1728, 9440, 9728
mask: 1440, 1728, 1440, 1728

(288, 288, 3) (288, 288, 3)


In [90]:
df

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_310000_6780000,25814,59.959247,68.316495,55.590067,102.711097,101.768423,81.375810,24.471098,21.347180,15.446078,35.955295,26.954412,26.620241
1,35_310000_6780144,9029,61.446672,69.951600,56.187950,109.235866,109.103646,89.719854,22.276342,19.769650,14.224425,37.310272,29.805334,35.249732
2,35_310000_6780288,20804,60.544751,68.187752,55.097049,89.229691,92.177486,70.350885,23.605553,20.746002,14.023302,24.341199,17.567141,16.055097
3,35_310000_6780432,18207,55.384522,64.959356,51.908222,83.817647,87.680863,66.795480,21.257001,19.739165,13.333501,25.222538,21.168019,16.170467
4,35_310000_6780576,53279,51.450759,59.564143,49.372642,69.553042,71.209911,61.290848,20.766370,18.975966,13.160716,37.028358,31.839744,29.169495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,35_314720_6780144,13422,71.420653,80.621368,64.499180,102.500273,110.221843,84.335836,24.116026,21.840670,14.384106,37.807663,29.140788,30.727219
176,35_314720_6780288,10567,68.826819,76.593641,61.701050,98.752725,104.663374,82.981693,27.162081,23.912048,16.343955,35.944498,27.509878,30.110999
177,35_314720_6780432,6829,64.564358,75.645629,59.419681,78.145306,95.070696,66.503672,23.851658,22.305197,16.905200,15.785723,14.373088,11.970231
178,35_314720_6780576,11292,65.134786,75.806854,58.481314,74.525052,90.080403,65.106361,21.902719,20.627260,13.820887,17.384981,16.187764,13.638917


In [127]:
import cv2

# Load your low-contrast image
image = cv2.imread('vignettes/rgb/rgb_305_6785/rgb_35_306000_6783512.jpg')
mask = cv2.imread('vignettes/mask/mask_305_6785/mask_35_306000_6783512.png')
image_to_equalize = cv2.imread('vignettes/rgb/rgb_305_6785/rgb_35_306000_6783512.jpg', cv2.IMREAD_GRAYSCALE)

# Apply histogram equalization
equalized_image = cv2.equalizeHist(image_to_equalize)

# Create a CLAHE object
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
# Apply CLAHE to the image
adap_equalized_image = clahe.apply(image_to_equalize)

In [ ]:
import cv2

# Load your low-contrast image
image = cv2.imread('low_contrast_image.jpg', cv2.IMREAD_GRAYSCALE)

# Create a CLAHE object
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Apply CLAHE to the image
equalized_image = clahe.apply(image)
